# Baseline: Transfer Learning with ResNet

This notebook implements a baseline using transfer learning with ResNet50 for the Dogs vs Cats classification problem.

In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from PIL import Image
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Check GPU availability
print(f"GPU available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# Set paths
train_dir = '/home/data/train'
test_dir = '/home/data/test'
sample_submission_path = '/home/data/sample_submission.csv'

# Load file names and labels
train_files = os.listdir(train_dir)
train_labels = [1 if 'dog' in f else 0 for f in train_files]  # 1=dog, 0=cat

print(f"Total training images: {len(train_files)}")
print(f"Dog images: {sum(train_labels)}")
print(f"Cat images: {len(train_labels) - sum(train_labels)}")

# Split data
X_train, X_val, y_train, y_val = train_test_split(
    train_files, train_labels, test_size=0.2, stratify=train_labels, random_state=42
)

print(f"Training set size: {len(X_train)}")
print(f"Validation set size: {len(X_val)}")

In [ ]:
# Define dataset class
class DogsCatsDataset(Dataset):
    def __init__(self, file_names, labels, data_dir, transform=None):
        self.file_names = file_names
        self.labels = labels
        self.data_dir = data_dir
        self.transform = transform
        
    def __len__(self):
        return len(self.file_names)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.data_dir, self.file_names[idx])
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
            
        label = self.labels[idx] if self.labels is not None else 0
        return image, label

# Define transforms
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
# Create data loaders
train_dataset = DogsCatsDataset(X_train, y_train, train_dir, transform=train_transform)
val_dataset = DogsCatsDataset(X_val, y_val, train_dir, transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)

print(f"Training batches: {len(train_loader)}")
print(f"Validation batches: {len(val_loader)}")

In [ ]:
# Load pre-trained ResNet50
model = models.resnet50(pretrained=True)

# Freeze early layers
for param in model.parameters():
    param.requires_grad = False

# Replace the final layer
num_features = model.fc.in_features
model.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(num_features, 1),
    nn.Sigmoid()
)

# Move to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

print(f"Model loaded on: {device}")
print(f"Total parameters: {sum(p.numel() for p in model.parameters())}")
print(f"Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad)}")

In [ ]:
# Define loss and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

# Training function
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    
    for images, labels in loader:
        images = images.to(device)
        labels = labels.float().unsqueeze(1).to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
    
    return running_loss / len(loader.dataset)

# Validation function
def validate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            labels = labels.float().unsqueeze(1).to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item() * images.size(0)
            all_preds.extend(outputs.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    val_loss = running_loss / len(loader.dataset)
    val_logloss = log_loss(all_labels, all_preds)
    
    return val_loss, val_logloss

In [ ]:
# Train the model
num_epochs = 5
best_logloss = float('inf')

for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    
    train_loss = train_epoch(model, train_loader, criterion, optimizer, device)
    val_loss, val_logloss = validate(model, val_loader, criterion, device)
    
    print(f"Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val LogLoss: {val_logloss:.4f}")
    
    if val_logloss < best_logloss:
        best_logloss = val_logloss
        torch.save(model.state_dict(), '/home/code/models/best_model.pth')
        print("Model saved!")
    
    print("-" * 50)

print(f"Best Validation LogLoss: {best_logloss:.4f}")

In [ ]:
# Load best model
model.load_state_dict(torch.load('/home/code/models/best_model.pth'))
model.eval()

# Generate predictions on test set
test_files = sorted(os.listdir(test_dir))
test_dataset = DogsCatsDataset(test_files, None, test_dir, transform=val_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

test_preds = []
with torch.no_grad():
    for images, _ in test_loader:
        images = images.to(device)
        outputs = model(images)
        test_preds.extend(outputs.cpu().numpy().flatten())

print(f"Generated predictions for {len(test_preds)} test images")
print(f"Sample predictions: {test_preds[:5]}")

In [ ]:
# Create submission file
submission = pd.DataFrame({
    'id': [int(f.split('.')[0]) for f in test_files],
    'label': test_preds
})

submission = submission.sort_values('id')
submission_path = '/home/submission/submission_001_baseline.csv'
submission.to_csv(submission_path, index=False)

print(f"Submission saved to: {submission_path}")
print(submission.head())
print(f"\nFinal validation LogLoss: {best_logloss:.4f}")